In [1]:
import pandas as pd
import numpy as np

data_url = 'http://vincentarelbundock.github.io/Rdatasets/csv/carData/Salaries.csv'
df = pd.read_csv(data_url, index_col=0)

display(df.head())
df.columns

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
1,Prof,B,19,18,Male,139750
2,Prof,B,20,16,Male,173200
3,AsstProf,B,4,3,Male,79750
4,Prof,B,45,39,Male,115000
5,Prof,B,40,41,Male,141500


Index(['rank', 'discipline', 'yrs.since.phd', 'yrs.service', 'sex', 'salary'], dtype='object')

In [2]:
# Populate custom INFO table
compactInfoView = pd.DataFrame()
compactInfoView['Total Values'] = df.count()
compactInfoView['Unique Values'] = df.nunique()
compactInfoView['Null Values'] = df.isna().sum()
compactInfoView['Data Type'] = df.dtypes

display (compactInfoView.sort_values(by='Data Type'))

,Total Values,Unique Values,Null Values,Data Type
yrs.since.phd,397,53,0,int64
yrs.service,397,52,0,int64
salary,397,371,0,int64
rank,397,3,0,object
discipline,397,2,0,object
sex,397,2,0,object


## Too many classes in Years

Our analysis in last step highlighted that two of our main columns for analysis have way too many classes. One way around is to introduce columns that convert multiple year classes into binary class, i.e., all over 25 years of service go into class 1 and those with under and equal to 25 years of service are categorized as class 0.

Two columns need this normalization:

1. **yrs.since.phd** (53 unique classes)
2. **yrs.service** (52 unique classes)

In [30]:
# if we apply any groupby operation, our results will be almost useless unless we are looking at a certain YEAR.
df.pivot_table(values='salary', index=['sex', 'rank'],
                     columns=['yrs.since.phd'], aggfunc=np.sum, fill_value=0)

yrs.since.phd         1       2       3        4       5       6       7   \
sex    rank                                                                 
Female AssocProf       0       0       0        0       0       0       0   
       AsstProf        0   72500  164500   157725  150500       0   63100   
       Prof            0       0       0        0       0       0       0   
Male   AssocProf       0       0       0        0       0  192408       0   
       AsstProf   325263  340916  477627  1425353  316806  334862  411889   
       Prof            0       0       0        0       0       0       0   

yrs.since.phd         8       9       10  ...      44      45      46     47  \
sex    rank                               ...                                  
Female AssocProf       0       0   77500  ...       0       0       0      0   
       AsstProf    78500       0   97032  ...       0       0       0      0   
       Prof            0       0       0  ...       0       0       0      0   
Male   AssocProf  376755  768229  770467  ...       0   70700       0      0   
       AsstProf   691141   73800       0  ...       0       0       0      0   
       Prof            0       0       0  ...  105000  673036  389411  89650   

yrs.since.phd        48      49     51      52     54      56  
sex    rank                                                    
Female AssocProf      0       0      0       0      0       0  
       AsstProf       0       0      0       0      0       0  
       Prof           0       0      0       0      0       0  
Male   AssocProf  90000   81800      0       0      0       0  
       AsstProf       0       0      0       0      0       0  
       Prof           0  353262  57800  107200  78162  263800  

[6 rows x 53 columns]

## STEP: Years <= 25 = 0 AND Years > 25 = 1

We create two new columns with the decided logic to facilitate our grouping and pivoting for better visibility and translation of data.

In [3]:
moddedDF = df.copy()
moddedDF['YEARS_OF_SERVICE'] = np.select([moddedDF['yrs.service']<= 25, moddedDF['yrs.service'] > 25], \
                                        choicelist=['yrs <= 25', 'yrs > 25'], default=np.nan)
moddedDF['YEARS_SINCE_PHD'] = np.select([moddedDF['yrs.since.phd']<= 25, moddedDF['yrs.since.phd'] > 25], \
                                        choicelist=['yrs <= 25', 'yrs > 25'], default=np.nan)

# moddedDF = moddedDF.drop('yrs.service', axis=1)

display (moddedDF.head())

moddedDF.pivot_table(values='salary', index=['sex', 'rank'],
                     columns=['YEARS_OF_SERVICE'], aggfunc=np.sum, fill_value=0)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary,YEARS_OF_SERVICE,YEARS_SINCE_PHD
1,Prof,B,19,18,Male,139750,yrs <= 25,yrs <= 25
2,Prof,B,20,16,Male,173200,yrs <= 25,yrs <= 25
3,AsstProf,B,4,3,Male,79750,yrs <= 25,yrs <= 25
4,Prof,B,45,39,Male,115000,yrs > 25,yrs > 25
5,Prof,B,40,41,Male,141500,yrs > 25,yrs > 25


YEARS_OF_SERVICE  yrs <= 25  yrs > 25
sex    rank                          
Female AssocProf     885128         0
       AsstProf      858549         0
       Prof         1822766    372651
Male   AssocProf    4685564    437400
       AsstProf     4553442         0
       Prof        19222158  12303806

## Calculate: [sex, rank, years_of_service] vise AVERAGE SALARY

In this section we use the encoded data and apply our first analytics on it.

#### LOGIC:
> This analytics make use of 3 columns to create the grouping and then use agg() method for 'SUM' and 'COUNT' to then compute AVERAGE_SALARY for each group.

In [4]:
sryGroup = moddedDF.groupby(['sex', 'rank', 'YEARS_OF_SERVICE'])

# Count number of observations in each group
display (sryGroup.size().reset_index(name='count'))

display (sryGroup.get_group(('Male', 'Prof', 'yrs <= 25')).head())

sryGroupAGG = sryGroup['salary'].agg(['sum', 'count'])

sryGroupAGG['AVG_SALARY'] = sryGroupAGG['sum']/sryGroupAGG['count']

display (sryGroupAGG)

,sex,rank,YEARS_OF_SERVICE,count
0,Female,AssocProf,yrs <= 25,10
1,Female,AsstProf,yrs <= 25,11
2,Female,Prof,yrs <= 25,15
3,Female,Prof,yrs > 25,3
4,Male,AssocProf,yrs <= 25,49
5,Male,AssocProf,yrs > 25,5
6,Male,AsstProf,yrs <= 25,56
7,Male,Prof,yrs <= 25,149
8,Male,Prof,yrs > 25,99


,rank,discipline,yrs.since.phd,yrs.service,sex,salary,YEARS_OF_SERVICE,YEARS_SINCE_PHD
1,Prof,B,19,18,Male,139750,yrs <= 25,yrs <= 25
2,Prof,B,20,16,Male,173200,yrs <= 25,yrs <= 25
7,Prof,B,30,23,Male,175000,yrs <= 25,yrs > 25
9,Prof,B,21,20,Male,119250,yrs <= 25,yrs <= 25
15,Prof,B,20,18,Male,104800,yrs <= 25,yrs <= 25


sum  count     AVG_SALARY
sex    rank      YEARS_OF_SERVICE                                
Female AssocProf yrs <= 25           885128     10   88512.800000
       AsstProf  yrs <= 25           858549     11   78049.909091
       Prof      yrs <= 25          1822766     15  121517.733333
                 yrs > 25            372651      3  124217.000000
Male   AssocProf yrs <= 25          4685564     49   95623.755102
                 yrs > 25            437400      5   87480.000000
       AsstProf  yrs <= 25          4553442     56   81311.464286
       Prof      yrs <= 25         19222158    149  129007.771812
                 yrs > 25          12303806     99  124280.868687

## Add calculated AVG_SALARY to dataframe

This crucial step is part of our data pipeline.

#### REASON:

> The computed column is then made part of our dataframe using ***JOIN*** based logic so that same can be stored in Data Mart for further Machine Learning implementation. We can't use ***np.select()*** because it will only update the cases we specify in condlist and that is problematic becauese we will have to specify all possible conditions in our dataframe to match it entirely with output column.

In [14]:
sryGroupAGG_DF = sryGroupAGG.reset_index() # to access data, we need to convert our index to columns

display (sryGroupAGG_DF)
# use np.select() here.
# .
# .
# .
# .
# . condlist[moddedDF['sex']]

# use join

# MERGE the two dataframes on 3 columns and MERGE() will automatically omit the duplicate columns from output result.
moddedDF_AVG = pd.merge(moddedDF, sryGroupAGG_DF, left_on=['sex', 'rank', 'YEARS_OF_SERVICE'], \
                   right_on=['sex', 'rank', 'YEARS_OF_SERVICE'], how='inner')

display (moddedDF_AVG.head())

,sex,rank,YEARS_OF_SERVICE,sum,count,AVG_SALARY
0,Female,AssocProf,yrs <= 25,885128,10,88512.800000
1,Female,AsstProf,yrs <= 25,858549,11,78049.909091
2,Female,Prof,yrs <= 25,1822766,15,121517.733333
3,Female,Prof,yrs > 25,372651,3,124217.000000
4,Male,AssocProf,yrs <= 25,4685564,49,95623.755102
5,Male,AssocProf,yrs > 25,437400,5,87480.000000
6,Male,AsstProf,yrs <= 25,4553442,56,81311.464286
7,Male,Prof,yrs <= 25,19222158,149,129007.771812
8,Male,Prof,yrs > 25,12303806,99,124280.868687


,rank,discipline,yrs.since.phd,yrs.service,sex,salary,YEARS_OF_SERVICE,YEARS_SINCE_PHD,sum,count,AVG_SALARY
0,Prof,B,19,18,Male,139750,yrs <= 25,yrs <= 25,19222158,149,129007.771812
1,Prof,B,20,16,Male,173200,yrs <= 25,yrs <= 25,19222158,149,129007.771812
2,Prof,B,30,23,Male,175000,yrs <= 25,yrs > 25,19222158,149,129007.771812
3,Prof,B,21,20,Male,119250,yrs <= 25,yrs <= 25,19222158,149,129007.771812
4,Prof,B,20,18,Male,104800,yrs <= 25,yrs <= 25,19222158,149,129007.771812


***

# Section 2 with custom Dataframes

In this section, we practice various GROUPING related workflow operations on custom dataframes.



In [4]:
'''
    testing groupby() operation
'''
dfTemp = pd.DataFrame({'author':['a', 'a', 'b'], 'subreddit':['sr1', 'sr2', 'sr2']})
display (dfTemp)

authorGrp = dfTemp.groupby('author')
# display (authorGrp.get_group('a'))


# ------ check if mean value for group is bigger than 0.5 then assign it 1, else 0.
traderDF = pd.DataFrame({'Buy/Sell': [1, 0, 1, 1, 0, 1, 0, 0],
                   'Trader': ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C']})

display (traderDF)

traderGroup = traderDF.groupby('Trader')
traderAggs = traderGroup['Buy/Sell'].agg(['count', 'sum'])
traderMeans = traderGroup['Buy/Sell'].mean()

traderAggs['Buy/Sell'] = np.select(condlist=[traderMeans > 0.5, traderMeans < 0.5], choicelist=[1, 0], default=np.nan)

display (traderMeans)
display (traderAggs)

,author,subreddit
0,a,sr1
1,a,sr2
2,b,sr2


,Buy/Sell,Trader
0,1,A
1,0,A
2,1,B
3,1,B
4,0,B
5,1,C
6,0,C
7,0,C


Trader
A    0.500000
B    0.666667
C    0.333333
Name: Buy/Sell, dtype: float64

,count,sum,Buy/Sell
Trader,,,
A,2,1,NaN
B,3,2,1.0
C,3,1,0.0


In [7]:
# PIVOT Table

df_pivot = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                          "bar", "bar", "bar", "bar"],
                    "B": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "C": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                          "large"],
                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df_pivot

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [15]:
# from pandas.pivot_table import pivot_table
# df.pivot_table('C', 'D', 'E', aggfunc=np.sum, margins=True)

table_dfPivot = df_pivot.pivot_table(values='D', index=['A', 'B'],
                     columns=['C'], aggfunc=np.sum, fill_value=0)
table_dfPivot

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [23]:
table_dfPivot = df_pivot.pivot_table(values='D', index=['A', 'C'],
                     columns=['B'], aggfunc=np.sum, fill_value=0)
table_dfPivot

B          one  two
A   C              
bar large    4    7
    small    5    6
foo large    4    0
    small    1    6